# Phase 4: Peri-Urban Growth Classification

## Objective
Identify districts undergoing peri-urban transformation using explainable,
rule-based logic derived from address update behavior and youth population signals.

## Input
- `data/processed/district_features.csv` (district × month)

## Output
- `data/final/peri_urban_labels.csv`

## Classification Rules (Applied in Priority Order)
| Priority | Label | Conditions |
|----------|-------|------------|
| 1 | Peri-Urban Growth | velocity > 0.12 AND youth_ratio > 0.38 AND growth_3m_consistent |
| 2 | Stable Urban | velocity >= 0.00 AND velocity <= 0.12 AND youth_ratio >= 0.38 |
| 3 | Rural Stable | velocity >= -0.05 AND velocity <= 0.00 |
| 4 | Declining | velocity < -0.05 |

## Growth Confidence Formula
```
growth_confidence = MIN(1.0, (address_velocity / 0.12) * 0.5 + (youth_ratio / 0.38) * 0.5)
```

## Cell 1: Import Libraries and Set Paths

In [1]:
"""
Import Required Libraries
-------------------------
- pandas: Data manipulation
- pathlib: Cross-platform file path handling
"""
import pandas as pd
from pathlib import Path

# Define project paths
PROJECT_ROOT = Path.cwd().parent
INPUT_PATH = PROJECT_ROOT / "data" / "processed" / "district_features.csv"
OUTPUT_PATH = PROJECT_ROOT / "data" / "final" / "peri_urban_labels.csv"

# Ensure output directory exists
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

print(f"✅ Libraries imported")
print(f"📂 Input: {INPUT_PATH}")
print(f"📂 Output: {OUTPUT_PATH}")

✅ Libraries imported
📂 Input: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\processed\district_features.csv
📂 Output: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\final\peri_urban_labels.csv


## Cell 2: Load Feature Data from Phase 2

In [2]:
"""
Load Feature Data
-----------------
Read the district-month feature table from Phase 2.
We need: address_velocity, youth_ratio, growth_3m_consistent
"""
df = pd.read_csv(INPUT_PATH)

# Convert growth_3m_consistent to boolean if stored as string
if df['growth_3m_consistent'].dtype == 'object':
    df['growth_3m_consistent'] = df['growth_3m_consistent'].map({'True': True, 'False': False})

print(f"✅ Loaded {len(df):,} rows from Phase 2 output")
print(f"\n📊 Columns: {list(df.columns)}")
print(f"\n📋 Key feature statistics:")
print(f"   address_velocity: min={df['address_velocity'].min():.4f}, max={df['address_velocity'].max():.4f}")
print(f"   youth_ratio: min={df['youth_ratio'].min():.4f}, max={df['youth_ratio'].max():.4f}")
print(f"   growth_3m_consistent: {df['growth_3m_consistent'].sum()} True values")
print(f"\n📋 Sample rows:")
df[['district_id', 'month', 'address_velocity', 'youth_ratio', 'growth_3m_consistent']].head(10)

✅ Loaded 240 rows from Phase 2 output

📊 Columns: ['district_id', 'month', 'address_velocity', 'migration_index', 'youth_ratio', 'digital_gap', 'growth_3m_consistent']

📋 Key feature statistics:
   address_velocity: min=-0.5745, max=1.7011
   youth_ratio: min=0.3859, max=0.4126
   growth_3m_consistent: 5 True values

📋 Sample rows:


,district_id,month,address_velocity,youth_ratio,growth_3m_consistent
0,GJ_AMD,2024-01,0.0000,0.4038,False
1,GJ_AMD,2024-02,0.5993,0.4038,False
2,GJ_AMD,2024-03,-0.1842,0.4038,False
3,GJ_AMD,2024-04,-0.1129,0.4038,False
4,GJ_AMD,2024-05,0.8165,0.4038,False
5,GJ_AMD,2024-06,-0.1249,0.4038,False
6,GJ_AMD,2024-07,-0.3278,0.4038,False
7,GJ_AMD,2024-08,0.3106,0.4038,False
8,GJ_AMD,2024-09,0.1677,0.4038,False
9,GJ_AMD,2024-10,-0.4915,0.4038,False


## Cell 3: Apply Peri-Urban Classification Rules

**Classification Logic (evaluated in priority order):**

1. **Peri-Urban Growth**: Districts showing sustained growth pattern
   - address_velocity > 0.12 (high growth rate)
   - youth_ratio > 0.38 (significant youth population)
   - growth_3m_consistent = TRUE (sustained 3-month growth)

2. **Stable Urban**: Mature urban districts with moderate growth
   - address_velocity between 0.00 and 0.12 (non-negative, moderate velocity)
   - youth_ratio >= 0.38 (significant youth population)

3. **Rural Stable**: Districts with minimal change
   - address_velocity between -0.05 and 0.00 (slight decline or flat)

4. **Declining**: Districts losing population
   - address_velocity < -0.05 (significant negative growth)

In [3]:
"""
Apply Peri-Urban Classification Rules
--------------------------------------
Each district-month is classified based on explicit, explainable rules.

Rule Priority (evaluated in order):
1. Peri-Urban Growth: High velocity + youth + sustained growth
2. Stable Urban: Moderate velocity (0 to 0.12) + youth >= 0.38
3. Rural Stable: Slight decline or flat velocity (-0.05 to 0)
4. Declining: Significant negative velocity (< -0.05)
"""

def classify_peri_urban(row) -> str:
    """
    Classify a district-month into one of 4 peri-urban categories.
    
    Args:
        row: DataFrame row with address_velocity, youth_ratio, growth_3m_consistent
    
    Returns:
        One of: 'Peri-Urban Growth', 'Stable Urban', 'Rural Stable', 'Declining'
    """
    velocity = row['address_velocity']
    youth = row['youth_ratio']
    growth_consistent = row['growth_3m_consistent']
    
    # Rule 1: Peri-Urban Growth (HIGHEST PRIORITY)
    # Districts with high address velocity, significant youth population,
    # AND sustained 3-month growth are classified as peri-urban growth zones.
    # These are areas transitioning from rural to urban.
    if velocity > 0.12 and youth > 0.38 and growth_consistent:
        return "Peri-Urban Growth"
    
    # Rule 2: Stable Urban
    # Districts with moderate velocity (between 0.00 and 0.12)
    # AND youth ratio >= 0.38 are mature urban centers.
    elif velocity >= 0.00 and velocity <= 0.12 and youth >= 0.38:
        return "Stable Urban"
    
    # Rule 3: Rural Stable
    # Districts with slight decline or flat velocity (between -0.05 and 0.00)
    # are stable rural areas with minimal population change.
    elif velocity >= -0.05 and velocity <= 0.00:
        return "Rural Stable"
    
    # Rule 4: Declining (LOWEST PRIORITY)
    # Districts with significant negative velocity (< -0.05)
    # are experiencing population decline or reverse migration.
    else:
        return "Declining"

# Apply classification to each row
df['peri_urban_label'] = df.apply(classify_peri_urban, axis=1)

print(f"✅ Peri-urban labels assigned")
print(f"\n📊 Label distribution:")
print(df['peri_urban_label'].value_counts().sort_index())
print(f"\n📋 Sample classified rows:")
df[['district_id', 'month', 'address_velocity', 'youth_ratio', 'growth_3m_consistent', 'peri_urban_label']].head(12)

✅ Peri-urban labels assigned

📊 Label distribution:
peri_urban_label
Declining            173
Peri-Urban Growth      1
Rural Stable          15
Stable Urban          51
Name: count, dtype: int64

📋 Sample classified rows:


,district_id,month,address_velocity,youth_ratio,growth_3m_consistent,peri_urban_label
0,GJ_AMD,2024-01,0.0000,0.4038,False,Stable Urban
1,GJ_AMD,2024-02,0.5993,0.4038,False,Declining
2,GJ_AMD,2024-03,-0.1842,0.4038,False,Declining
3,GJ_AMD,2024-04,-0.1129,0.4038,False,Declining
4,GJ_AMD,2024-05,0.8165,0.4038,False,Declining
5,GJ_AMD,2024-06,-0.1249,0.4038,False,Declining
6,GJ_AMD,2024-07,-0.3278,0.4038,False,Declining
7,GJ_AMD,2024-08,0.3106,0.4038,False,Declining
8,GJ_AMD,2024-09,0.1677,0.4038,False,Declining
9,GJ_AMD,2024-10,-0.4915,0.4038,False,Declining


## Cell 4: Compute Growth Confidence Score

**Formula (from specification):**
```
growth_confidence = MIN(1.0, (address_velocity / 0.12) * 0.5 + (youth_ratio / 0.38) * 0.5)
```

**Notes:**
- This is NOT a probability, only a confidence indicator
- Values are clamped to [0, 1]
- Two equal components: normalized velocity and normalized youth ratio

In [4]:
"""
Compute Growth Confidence Score
--------------------------------
A normalized score between 0 and 1 that indicates how confident we are
that a district is experiencing growth transformation.

Formula (from specification):
growth_confidence = MIN(1.0, (address_velocity / 0.12) * 0.5 + (youth_ratio / 0.38) * 0.5)

Components:
- 50% weight: Normalized address velocity (velocity / 0.12)
- 50% weight: Normalized youth ratio (youth_ratio / 0.38)

NOTE: This is NOT a probability, only a confidence indicator.
Values are clamped to [0, 1].
"""

def compute_growth_confidence(row) -> float:
    """
    Compute growth confidence score for a district-month.
    
    Args:
        row: DataFrame row with address_velocity, youth_ratio
    
    Returns:
        float: Confidence score clamped to [0, 1]
    """
    velocity = row['address_velocity']
    youth = row['youth_ratio']
    
    # Compute raw confidence score using the specified formula
    # (address_velocity / 0.12) * 0.5 + (youth_ratio / 0.38) * 0.5
    velocity_component = (velocity / 0.12) * 0.5
    youth_component = (youth / 0.38) * 0.5
    
    raw_confidence = velocity_component + youth_component
    
    # Clamp to [0, 1] as specified: MIN(1.0, raw_confidence)
    # Also ensure minimum is 0 (no negative confidence)
    return max(0.0, min(1.0, raw_confidence))

# Apply confidence calculation to each row
df['growth_confidence'] = df.apply(compute_growth_confidence, axis=1)

# Round to 4 decimal places for clean output
df['growth_confidence'] = df['growth_confidence'].round(4)

print(f"✅ Growth confidence scores computed")
print(f"\n📊 Score statistics:")
print(f"   Min: {df['growth_confidence'].min():.4f}")
print(f"   Max: {df['growth_confidence'].max():.4f}")
print(f"   Mean: {df['growth_confidence'].mean():.4f}")
print(f"\n📊 Mean confidence by label:")
print(df.groupby('peri_urban_label')['growth_confidence'].mean().round(4))
print(f"\n📋 Sample rows with confidence:")
df[['district_id', 'month', 'address_velocity', 'youth_ratio', 'growth_confidence', 'peri_urban_label']].head(12)

✅ Growth confidence scores computed

📊 Score statistics:
   Min: 0.0000
   Max: 1.0000
   Mean: 0.5157

📊 Mean confidence by label:
peri_urban_label
Declining            0.4886
Peri-Urban Growth    1.0000
Rural Stable         0.3976
Stable Urban         0.6330
Name: growth_confidence, dtype: float64

📋 Sample rows with confidence:


,district_id,month,address_velocity,youth_ratio,growth_confidence,peri_urban_label
0,GJ_AMD,2024-01,0.0000,0.4038,0.5313,Stable Urban
1,GJ_AMD,2024-02,0.5993,0.4038,1.0000,Declining
2,GJ_AMD,2024-03,-0.1842,0.4038,0.0000,Declining
3,GJ_AMD,2024-04,-0.1129,0.4038,0.0609,Declining
4,GJ_AMD,2024-05,0.8165,0.4038,1.0000,Declining
5,GJ_AMD,2024-06,-0.1249,0.4038,0.0109,Declining
6,GJ_AMD,2024-07,-0.3278,0.4038,0.0000,Declining
7,GJ_AMD,2024-08,0.3106,0.4038,1.0000,Declining
8,GJ_AMD,2024-09,0.1677,0.4038,1.0000,Declining
9,GJ_AMD,2024-10,-0.4915,0.4038,0.0000,Declining


## Cell 5: Prepare Final Output Schema

In [5]:
"""
Prepare Final Output
--------------------
Select only the required columns in the exact schema order.

Output Schema:
- district_id (string)
- month (string, YYYY-MM)
- peri_urban_label (string)
- growth_confidence (float, 0-1)
"""

# Define exact output schema
OUTPUT_COLUMNS = [
    'district_id',
    'month',
    'peri_urban_label',
    'growth_confidence'
]

# Select only required columns
df_output = df[OUTPUT_COLUMNS].copy()

# Sort by district_id and month for deterministic output
df_output = df_output.sort_values(['district_id', 'month']).reset_index(drop=True)

print(f"✅ Final output prepared: {len(df_output):,} rows")
print(f"\n📊 Columns: {list(df_output.columns)}")
print(f"\n📋 Data types:")
print(df_output.dtypes)
print(f"\n📋 First 12 rows:")
df_output.head(12)

✅ Final output prepared: 240 rows

📊 Columns: ['district_id', 'month', 'peri_urban_label', 'growth_confidence']

📋 Data types:
district_id           object
month                 object
peri_urban_label      object
growth_confidence    float64
dtype: object

📋 First 12 rows:


,district_id,month,peri_urban_label,growth_confidence
0,GJ_AMD,2024-01,Stable Urban,0.5313
1,GJ_AMD,2024-02,Declining,1.0000
2,GJ_AMD,2024-03,Declining,0.0000
3,GJ_AMD,2024-04,Declining,0.0609
4,GJ_AMD,2024-05,Declining,1.0000
5,GJ_AMD,2024-06,Declining,0.0109
6,GJ_AMD,2024-07,Declining,0.0000
7,GJ_AMD,2024-08,Declining,1.0000
8,GJ_AMD,2024-09,Declining,1.0000
9,GJ_AMD,2024-10,Declining,0.0000


## Cell 6: Data Quality Validation

In [6]:
"""
Data Quality Checks
-------------------
Validate all required constraints before export.
"""

checks_passed = 0
total_checks = 6

# Check 1: No NaN values
nan_count = df_output.isna().sum().sum()
if nan_count == 0:
    print("✅ Check 1: No NaN values")
    checks_passed += 1
else:
    print(f"❌ Check 1: Found {nan_count} NaN values")

# Check 2: growth_confidence is between 0 and 1
gc_valid = (df_output['growth_confidence'] >= 0).all() and (df_output['growth_confidence'] <= 1).all()
if gc_valid:
    print("✅ Check 2: growth_confidence in range [0, 1]")
    checks_passed += 1
else:
    print(f"❌ Check 2: growth_confidence out of range")

# Check 3: Valid peri_urban_label values
valid_labels = {'Peri-Urban Growth', 'Stable Urban', 'Rural Stable', 'Declining'}
actual_labels = set(df_output['peri_urban_label'].unique())
if actual_labels.issubset(valid_labels):
    print(f"✅ Check 3: Valid label values: {actual_labels}")
    checks_passed += 1
else:
    print(f"❌ Check 3: Invalid labels found: {actual_labels - valid_labels}")

# Check 4: peri_urban_label is non-null
if df_output['peri_urban_label'].notna().all():
    print("✅ Check 4: peri_urban_label is non-null")
    checks_passed += 1
else:
    print("❌ Check 4: peri_urban_label has null values")

# Check 5: Correct column count (exactly 4)
if len(df_output.columns) == 4:
    print("✅ Check 5: Exactly 4 columns (no extra columns)")
    checks_passed += 1
else:
    print(f"❌ Check 5: Expected 4 columns, got {len(df_output.columns)}")

# Check 6: Row count matches input
expected_rows = len(df)
if len(df_output) == expected_rows:
    print(f"✅ Check 6: Row count matches input ({expected_rows} rows)")
    checks_passed += 1
else:
    print(f"❌ Check 6: Row count mismatch. Expected {expected_rows}, got {len(df_output)}")

print(f"\n{'='*50}")
if checks_passed == total_checks:
    print(f"✅ ALL {total_checks} VALIDATION CHECKS PASSED")
else:
    print(f"⚠️ {checks_passed}/{total_checks} checks passed")

✅ Check 1: No NaN values
✅ Check 2: growth_confidence in range [0, 1]
✅ Check 3: Valid label values: {'Rural Stable', 'Declining', 'Stable Urban', 'Peri-Urban Growth'}
✅ Check 4: peri_urban_label is non-null
✅ Check 5: Exactly 4 columns (no extra columns)
✅ Check 6: Row count matches input (240 rows)

✅ ALL 6 VALIDATION CHECKS PASSED


## Cell 7: Export to CSV

In [7]:
"""
Export to CSV
-------------
Save the peri-urban classification data in CSV format.
"""

# Export to CSV (no index column)
df_output.to_csv(OUTPUT_PATH, index=False)

print(f"✅ CSV exported to: {OUTPUT_PATH}")
print(f"\n📊 Export Summary:")
print(f"   Total rows: {len(df_output):,}")
print(f"   Districts: {df_output['district_id'].nunique()}")
print(f"   Months: {df_output['month'].nunique()}")
print(f"\n📊 Label distribution:")
for label, count in df_output['peri_urban_label'].value_counts().sort_index().items():
    pct = count / len(df_output) * 100
    print(f"   {label}: {count} ({pct:.1f}%)")
print(f"\n📋 Schema:")
for col in df_output.columns:
    sample_val = df_output[col].iloc[0]
    print(f"   {col}: {df_output[col].dtype} (e.g., {sample_val})")
print(f"\n📄 First 10 rows:")
df_output.head(10)

✅ CSV exported to: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\final\peri_urban_labels.csv

📊 Export Summary:
   Total rows: 240
   Districts: 20
   Months: 12

📊 Label distribution:
   Declining: 173 (72.1%)
   Peri-Urban Growth: 1 (0.4%)
   Rural Stable: 15 (6.2%)
   Stable Urban: 51 (21.2%)

📋 Schema:
   district_id: object (e.g., GJ_AMD)
   month: object (e.g., 2024-01)
   peri_urban_label: object (e.g., Stable Urban)
   growth_confidence: float64 (e.g., 0.5313)

📄 First 10 rows:


,district_id,month,peri_urban_label,growth_confidence
0,GJ_AMD,2024-01,Stable Urban,0.5313
1,GJ_AMD,2024-02,Declining,1.0000
2,GJ_AMD,2024-03,Declining,0.0000
3,GJ_AMD,2024-04,Declining,0.0609
4,GJ_AMD,2024-05,Declining,1.0000
5,GJ_AMD,2024-06,Declining,0.0109
6,GJ_AMD,2024-07,Declining,0.0000
7,GJ_AMD,2024-08,Declining,1.0000
8,GJ_AMD,2024-09,Declining,1.0000
9,GJ_AMD,2024-10,Declining,0.0000


## Phase 4 Complete

### Output File
`data/final/peri_urban_labels.csv`

### Schema
| Field | Type | Description |
|-------|------|-------------|
| district_id | string | District identifier |
| month | string | YYYY-MM format |
| peri_urban_label | string | Classification label |
| growth_confidence | float | Confidence score (0-1) |

### Classification Rules Applied (Priority Order)
1. **Peri-Urban Growth**: velocity > 0.12 AND youth > 0.38 AND 3m consistent
2. **Stable Urban**: velocity >= 0 AND velocity <= 0.12 AND youth >= 0.38
3. **Rural Stable**: velocity >= -0.05 AND velocity <= 0.00
4. **Declining**: velocity < -0.05

### Growth Confidence Formula
```
growth_confidence = MIN(1.0, (address_velocity / 0.12) * 0.5 + (youth_ratio / 0.38) * 0.5)
```

### Next Phase
Phase 5 will add digital exclusion risk scores.